In [3]:
import pandas as pd
import re
import string
import unidecode
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
import nltk

In [4]:
nltk.download('punkt')
nltk.download('stopwords')
stemmer = SnowballStemmer('spanish')
pd.set_option('max_colwidth', 1000)

[nltk_data] Downloading package punkt to /Users/fernandoa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fernandoa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
tweets = pd.read_csv('assets/tweets.csv', encoding='utf-8', sep=',')
# We just select textual content for the moment. Other columns shall be used later on
tweets = tweets[['Text', 'Candidate']].copy()
initial_size = len(tweets.index)
tweets.dropna(inplace=True)
tweets.rename({'Candidate': 'Label'}, axis='columns', inplace=True)
tweets['Label'] = tweets['Label'].map({'Petro': 'left', 'Fajardo': 'center', 'Uribe': 'right'})

In [6]:
tweets.head()

,Text,Label
0,"#Ahora realizamos recorrido de seguridad en Santa Monica, sector Toberin, localidad de #Usaquén. \n\nJunto \n@PoliciaBogota\n, \n@Uaesp\n, \n@SeguridadBOG\n, \n@SectorMovilidad\n revisamos condiciones del sector. \n\nImportantes compromisos: instalación de luminarias y recuperación espacio público.",right
1,"Hoy, en caravana de moteros junto a \n@jjUscategui\n, rendimos homenaje a los héroes de nuestra Fuerza Pública asesinados este año. \n\nQue el rugido de estas motos sirva de llamado para que cese todo acto de violencia contra nuestros militares y policías. \n\n¡No están solos!",right
2,"Acuerdo La Habana obliga al PENIS (Sustitución droga);\n\nCada familia recibe 35 mill en 5 años. Total 100 mil flias y 3,5 bill.\n\nResultado, crecen cultivos.\n\nDenunciar y reformar: condicionar a que por esfuerzo colectivo no haya droga en el entorno.",right
3,"El POT una mentira más de Claudia López. \n\nLa meta es generar cerca de 171 hectáreas de espacio público, pero ¿cómo lo hará? sí habilita más de 6.900 hectáreas para la renovación urbana. \n\nEntre más aumenta la densidad, menos espacio público habrá.\n#PilasConElPOT",right
4,#UribeConLaVerdad - Sobre el Caso de Cajamarca: “El Ministerio de Defensa va a apersonarse de los legítimos intereses de todas estas personas llenas de dolor” \n@AlvaroUribeVel\n el 12 de abril de 2004,right


In [7]:
diff = initial_size - len(tweets.index)
print(f'{diff} records were lost; that is {diff/initial_size}% of the dataset.')

4 records were lost; that is 0.0013333333333333333% of the dataset.


In [8]:
def depure(text):
    text = text.lower()
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text) # Remove links and url's
    text = re.sub('#\S+', ' ', text) # Remove hashtags
    text = re.sub('@\S+', ' ', text) # Remove Twitter mentions
    text = re.sub('\w*\d\w*', ' ', text) # Remove words with numbers
    text = unidecode.unidecode(text) # More importantly, remove accents
    text = re.sub(r'[^\w\s]', ' ', text) # Remove puntuaction
    text = re.sub('\n', ' ', text) # Remove filthy, inconvinient new lines >:)))
    return text

In [9]:
spanish_stopwords = nltk.corpus.stopwords.words('spanish')
def remove_stopwords(words):
    return [word for word in words if word not in spanish_stopwords]

In [10]:
def stem(words):
    return [stemmer.stem(word) for word in words]

In [11]:
def process(text):
    depured_text = depure(text)
    tokens = word_tokenize(depured_text)
    non_stopwords = remove_stopwords(tokens)
    stemmed = stem(non_stopwords)
    return stemmed

In [12]:
tweets['Text'] = tweets['Text'].apply(process)

In [13]:
tweets.head()

,Text,Label
0,"[realiz, recorr, segur, sant, monic, sector, toberin, local, junt, revis, condicion, sector, import, compromis, instalacion, luminari, recuperacion, espaci, public]",right
1,"[hoy, caravan, moter, junt, rend, homenaj, her, fuerz, public, asesin, ano, rug, mot, sirv, llam, ces, acto, violenci, militar, polici, estan, sol]",right
2,"[acuerd, haban, oblig, penis, sustitucion, drog, cad, famili, recib, mill, anos, total, mil, fli, bill, result, crec, cultiv, denunci, reform, condicion, esfuerz, colect, drog, entorn]",right
3,"[pot, mentir, mas, claudi, lopez, met, gener, cerc, hectar, espaci, public, har, si, habilit, mas, hectar, renovacion, urban, mas, aument, densid, men, espaci, public, habr]",right
4,"[cas, cajamarc, ministeri, defens, va, aperson, legit, interes, tod, person, llen, dolor, abril]",right
